## PAIR PROGRAMMING LIMPIEZA III

### Valores Nulos Sklearn
---

En este caso trabajaremos con el dataframe que limpiamos en el ejercicio de pair de **Limpieza III** donde limpiabamos las columnas de species, age, etc. y eliminabamos los outliers. Hoy volveremos a gestionar valores nulos, pero en este caso usaremos los métodos de imputación de sklearn.

Antes de seguir, recordamos las preguntas que nos planteamos al principio del pair programming de EDA para dirigir nuestro análisis.

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

Actualizamos esta tabla ya que en el ejercicio de pair de Limpieza I y II ya dejamos algunas columnas limpitas:


- age ✔️ la limpiamos en el pair de Limpieza II
- species ✔️ la limpiamos en el pair de Limpieza II
- country ✔️ la limpiamos en el pair de Limpieza II
- fatal ✔️ la limpiamos en el pair de Pandas V
- year ✔️ es una columna de tipo float deberíamos convertirla a integer
- sex ✔️ la limpiamos en el pair de Pandas V
- fecha ✔️ la limpiamos en el pair de Pandas V

Es el momento de ponernos a trabajar con los valores nulos de nuevo 💪🏽. A lo largo de este ejercicio de pair programming vamos a intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos de skelarn intentemos aplicarlos todos. Manos a la obra!

**EJERCICIO 1**

Es el momento de eliminar los nulos:

- Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

- Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.
    💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

- Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

- Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.

- Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

- ¿Podríais explicar qué diferencia hay entre estos dos últimos métodos?

EJERCICIO 2

Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair